In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as an
from IPython.display import display, HTML

In [2]:
# animation class
class animate_imshow():
    def __init__(self, arrays, dt=1, t0=1):
        """
        Array is a list of 2d arrays or a 3d array with time on axis=0.
        Note in jupyter the animation needs to be done"
        """ 
        # setup data
        self.arr = arrays
        self.Nt = len(arrays)
        self.Nx = len(arrays[0])
        self.dt = dt
        self.t0 = t0

        # setup the figure
        self.fig = plt.figure(figsize=(6, 6))
        self.im = plt.imshow(arrays[0], animated=True, cmap='viridis')
        self.ax = plt.gca()
        self.text = self.ax.text(0.02*self.Nx, 0.04*self.Nx, 'Time: ' + str(t0), color='w')
        
        # settings
        self.delay = 10
        self.title = "title"
        self.xlabel = "xlabel"
        self.ylabel = "ylabel"

        # run the animation
    def run(self, html = False):
        self.ax.set(title=self.title, xlabel=self.xlabel, ylabel=self.ylabel)
        self.ani = an.FuncAnimation(self.fig, self.update, frames=range(self.Nt),
                                    interval=self.delay, blit=True)
        
        if html: # use this if in ipython/jupyter notebooks
            plt.close(self.fig)
            self.jshtml = self.ani.to_jshtml()
            display(HTML(self.jshtml))
    
    def save_html(self, filename = "output.html"):
        if not hasattr(self, 'jshtml'):
            self.jshtml = self.ani.to_jshtml()
        f = open(filename, 'w')
        f.write(self.jshtml)
        f.close()
            
    def update(self, f):
        self.im.set_array(self.arr[f])
        time = int(f * self.dt + self.t0)
        self.text.set_text('Time: %i' % time)
        return self.im, self.text

In [17]:
def load_results(directory, node, start, end):
    
    # check directory is real
    assert os.path.isdir(directory), "Given directory does not exist"
    
    # generate all file names
    files = [directory + str(i) + "-" + str(node) + ".csv" for i in range(iter_start, iter_end + 1)]

    # check all files exist
    assert all([os.path.isfile(f) for f in files]), "One or more iteration files does not exist"
    
    # load the data and check the dimensions match
    results = [np.loadtxt(f, dtype=int, delimiter=',') for f in files]
    assert all([results[0].shape == r.shape for r in results]), "Save files have incorrect dimensions"
    
    return results

In [14]:
# Load the results
directory = "../vs_gol/Debug/test_nonsquare/"
node = "all"
iter_start = 0
iter_end = 200

# check directory is real
assert os.path.isdir(directory), "Given directory does not exist"

# generate all file names
files = [directory + str(i) + "-" + str(node) + ".csv" for i in range(iter_start, iter_end + 1)]
#display(files)

# check all files exist
assert all([os.path.isfile(f) for f in files]), "One or more iteration files does not exist"

# load the data and check the dimensions match
results = [np.loadtxt(f, dtype=int, delimiter=',') for f in files]
#display([r.shape for r in results])
assert all([results[0].shape == r.shape for r in results]), "Save files have incorrect dimensions"
#display(results)

print("Results Loaded")

Results Loaded


In [20]:
# make the animation
directory = "../vs_gol/Debug/test_nonsquare/"
node = "all"
iter_start = 0
iter_end = 200
results = load_results(directory, node, iter_start, iter_end)

gol_ani = animate_imshow(results, 1, iter_start)
gol_ani.delay = 100
gol_ani.run(html=True)
gol_ani.save_html(directory + "_" + str(node) + "-" + str(iter_start) + "-" + str(iter_end) + ".html")